In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import random
import crossval #this is the file Eli wrote



from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV, train_test_split
import shap
import lightgbm as lgb 

pd.set_option('max_colwidth', 80)

In [ ]:
df = pd.read_csv('../../tacoma_ml_data.csv') 
df.head()

In [ ]:
param_list = {
      'n_estimators': [50],
      'max_depth': [30]
    } 
param = {}
for key in param_list.keys():
    val = random.choice(param_list[key])
    param[key] = val

In [ ]:
# Used to build a list containing dictionaries w every compination of params
# from param list, start with a list containing one dict (src), 
# then call for each param (cross, keyword)
def cross_params(src, cross, keyword):
    combos = []
    for d in src:
        for val in cross:
            cpy = d.copy()
            cpy[keyword] = val
            combos.append(cpy)
    return combos

# Uses cross_params to get list of all parameter combinations 
def get_param_combos(param_list):
    param_combos = [{}]
    for kw in param_list:
        param_combos = cross_params(param_combos, param_list[kw], kw)
    return param_combos

In [ ]:
def rand_param_search(data, param_list, classifier_fn, n_val=2, threshold=0.5, print_results=True):
    """

    """
    best_score = -1.0 
    best_param = {}
    param_combos = get_param_combos(param_list)
    for param in param_combos:
        print("using the following parameters:")
        print(param)
        avgs = crossval.crossval_by_year(data, classifier_fn, param, n_val, threshold, print_results)
        print(avgs[0])
        print(avgs[1])
        print(avgs[2])
        print()
        if avgs[0] > best_score:
            best_param = param
            best_score = avgs[0]
    
    
    return best_param, best_score

In [ ]:
best_score, best_param = rand_param_search(df, param_list, RandomForestClassifier)
print(best_param)
print(best_score)